# Arbres de décision

# Importation des modules

In [124]:
from typing import List, Dict

# pour la récupération des données depuis les fichiers XML
from lxml import etree
from preTraitements.xml import get_X_Y_from_root
from preTraitements.xml import get_tree_root_from_file

# pour la vectorisation et la classification
from sklearn import tree
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer

# validation
from sklearn.model_selection import cross_val_score

import spacy
from collections import Counter


nlp = spacy.load("fr_core_news_md")

# Pré-traitement

## Chargement des données depuis les XML

In [98]:
tree_train, root_train = get_tree_root_from_file("./corpus/train_deft09_parlement_appr.xml/deft09_parlement_appr_fr.xml")
X_train, y_train = get_X_Y_from_root(root_train)

tree_test, root_test = get_tree_root_from_file("./corpus/deft09_parlement_test.xml/deft09_parlement_test_fr.xml")
X_test, y_test = get_X_Y_from_root(root_test)

## Mise en forme des données

In [108]:
def count_tokens(text: List[List[str]], nlp) -> List[Dict[str, int]]:
    """Fonction qui, à partir d'une liste de listes de string, renvoie une liste de liste de dictionnaires
    Chaque dictionnaire correspond au nombre d'occurrences de chaque token pour chaque sous-liste de strings.
    La tokenization ne prend pas en compte les stop words, la poncutation et les espaces.

    Args:
        text (List[List[str]]): une liste de listes de string
        nlp (spacy.lang.fr.French): un modèle de spacy

    Returns:
        List[Dict[str, int]]: une liste de liste de dictionnaires tels que `dico[token]=nb_tokens`
    """
    return [dict(Counter(token.text for token in nlp(enonce) if not (token.is_stop or token.is_punct or token.is_space))) for enonce in text]

In [109]:
counters = count_tokens(X_train, nlp) # on tokenize et on compte le nombre d'occurrences de chaque token pour chaque énoncé

# transformation des dictionnaires en vecteurs
vec = DictVectorizer() 
X_train_vec = vec.fit_transform(counters)

# Classification

In [115]:
clf = tree.DecisionTreeClassifier() # création du classifieur
clf = clf.fit(X_train_vec, y_train) # entraînement

feature_names = vec.feature_names_ # récupération du nom des features

In [120]:
print(cross_val_score(clf, X_train_vec, y_train,cv=10)) # cross-validation

[0.33402168 0.33040785 0.33144037 0.32627775 0.35776975 0.33298916
 0.33557047 0.35105834 0.34641198 0.32885906]


In [121]:
feature_importances = clf.feature_importances_ # features les plus décisifs pour le classifieur
feature_importances_sorted = sorted(zip(feature_importances, feature_names), reverse=True) # on le trie par importance

for score, word in feature_importances_sorted[:100]: # on affiche le nom des features avec leur score pour les 100 premiers
    print(f"{word}\t{score}")

Monsieur	0.009952128232256482
libéraux	0.008066617359011105
Madame	0.006286814415960344
été	0.005710144638229512
Commission	0.005230304188553939
Parlement	0.004904703890456348
question	0.004447379243154804
Mesdames	0.004378333990652465
Union	0.0041933222158018185
rapport	0.004087112278552198
voudrais	0.004032932008524491
européenne	0.0039772699924455306
bien	0.003890237820298784
faut	0.003795906626577938
M.	0.0036391699590442445
devons	0.0036121148420693054
politique	0.0035827796170060803
retraités	0.0035797739839431414
pays	0.003522639710684918
Europe	0.0034875844045781797
non	0.0034557021414133673
groupe	0.0034006493720006737
européen	0.003129413153684946
travailleurs	0.003090027478159047
droit	0.0030762697298877244
droits	0.0029829361436590086
problème	0.0029712189149613214
faire	0.002936074190464172
UE	0.0028197263580555
fois	0.0028043707502507192
membres	0.0027892191078667822
cas	0.002763690280647359
développement	0.0027441104811562365
États-Unis	0.0027244114207357643
Conseil	0.00

## Pour plotter l'arbre de décision

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,7))
tree.plot_tree(clf, feature_names=feature_names)

# sauvegarde du plot dans un SVG
plt.savefig('decision_tree.svg')
